In [9]:
%load_ext autoreload
%autoreload 2
import sys
# sys.path.insert(0, '..')

import os
import time

from torch.utils.data import DataLoader

from dataloaders.nyu_v2_dataloader import NYU_v2
# from dataloaders.cityscapes_dataloader import CityScapes
# from dataloaders.taskonomy_dataloader import Taskonomy
from envs.blockdrop_env import BlockDropEnv
import torch
from utils.util import makedir, print_separator, create_path, print_yaml, should, fix_random_seed
import numpy as np
from sklearn.metrics import confusion_matrix
# from tqdm import tqdm

import argparse
import yaml

In [10]:
def eval(environ, dataloader, tasks, policy=False, num_train_layers=None, hard_sampling=False, num_seg_cls=-1,
         eval_iter=10):
    batch_size = []
    records = {}
    val_metrics = {}
    if 'seg' in tasks:
        assert (num_seg_cls != -1)
        records['seg'] = {'mIoUs': [], 'pixelAccs': [],  'errs': [], 'conf_mat': np.zeros((num_seg_cls, num_seg_cls)),
                          'labels': np.arange(num_seg_cls)}
    if 'sn' in tasks:
        records['sn'] = {'cos_similaritys': []}
    if 'depth' in tasks:
        records['depth'] = {'abs_errs': [], 'rel_errs': [], 'sq_rel_errs': [], 'ratios': [], 'rms': [], 'rms_log': []}
    if 'keypoint' in tasks:
        records['keypoint'] = {'errs': []}
    if 'edge' in tasks:
        records['edge'] = {'errs': []}

    with torch.no_grad():
        for batch_idx, batch in enumerate(tqdm(dataloader)):
            if eval_iter != -1:
                if batch_idx > eval_iter:
                    break

            environ.set_inputs(batch)
            # seg_pred, seg_gt, pixelAcc, cos_similarity = environ.val(policy, num_train_layers, hard_sampling)
            metrics = environ.val2(policy, num_train_layers, hard_sampling)

            # environ.networks['mtl-net'].task1_logits
            # mIoUs.append(mIoU)
            if 'seg' in tasks:
                new_mat = confusion_matrix(metrics['seg']['gt'], metrics['seg']['pred'], records['seg']['labels'])
                assert (records['seg']['conf_mat'].shape == new_mat.shape)
                records['seg']['conf_mat'] += new_mat
                records['seg']['pixelAccs'].append(metrics['seg']['pixelAcc'])
                records['seg']['errs'].append(metrics['seg']['err'])
            if 'sn' in tasks:
                records['sn']['cos_similaritys'].append(metrics['sn']['cos_similarity'])
            if 'depth' in tasks:
                records['depth']['abs_errs'].append(metrics['depth']['abs_err'])
                records['depth']['rel_errs'].append(metrics['depth']['rel_err'])
                records['depth']['sq_rel_errs'].append(metrics['depth']['sq_rel_err'])
                records['depth']['ratios'].append(metrics['depth']['ratio'])
                records['depth']['rms'].append(metrics['depth']['rms'])
                records['depth']['rms_log'].append(metrics['depth']['rms_log'])
            if 'keypoint' in tasks:
                records['keypoint']['errs'].append(metrics['keypoint']['err'])
            if 'edge' in tasks:
                records['edge']['errs'].append(metrics['edge']['err'])
            batch_size.append(len(batch['img']))

    # overall_mIoU = (np.array(mIoUs) * np.array(batch_size)).sum() / sum(batch_size)
    if 'seg' in tasks:
        val_metrics['seg'] = {}
        jaccard_perclass = []
        for i in range(num_seg_cls):
            if not records['seg']['conf_mat'][i, i] == 0:
                jaccard_perclass.append(records['seg']['conf_mat'][i, i] / (np.sum(records['seg']['conf_mat'][i, :]) +
                                                                            np.sum(records['seg']['conf_mat'][:, i]) -
                                                                            records['seg']['conf_mat'][i, i]))

        val_metrics['seg']['mIoU'] = np.sum(jaccard_perclass) / len(jaccard_perclass)

        val_metrics['seg']['Pixel Acc'] = (np.array(records['seg']['pixelAccs']) * np.array(batch_size)).sum() / sum(
            batch_size)

        val_metrics['seg']['err'] = (np.array(records['seg']['errs']) * np.array(batch_size)).sum() / sum(batch_size)

    if 'sn' in tasks:
        val_metrics['sn'] = {}
        overall_cos = np.clip(np.concatenate(records['sn']['cos_similaritys']), -1, 1)

        angles = np.arccos(overall_cos) / np.pi * 180.0
        val_metrics['sn']['cosine_similarity'] = overall_cos.mean()
        val_metrics['sn']['Angle Mean'] = np.mean(angles)
        val_metrics['sn']['Angle Median'] = np.median(angles)
        val_metrics['sn']['Angle RMSE'] = np.sqrt(np.mean(angles ** 2))
        val_metrics['sn']['Angle 11.25'] = np.mean(np.less_equal(angles, 11.25)) * 100
        val_metrics['sn']['Angle 22.5'] = np.mean(np.less_equal(angles, 22.5)) * 100
        val_metrics['sn']['Angle 30'] = np.mean(np.less_equal(angles, 30.0)) * 100
        val_metrics['sn']['Angle 45'] = np.mean(np.less_equal(angles, 45.0)) * 100

    if 'depth' in tasks:
        val_metrics['depth'] = {}
        records['depth']['abs_errs'] = np.stack(records['depth']['abs_errs'], axis=0)
        records['depth']['rel_errs'] = np.stack(records['depth']['rel_errs'], axis=0)
        records['depth']['sq_rel_errs'] = np.stack(records['depth']['sq_rel_errs'], axis=0)
        records['depth']['ratios'] = np.concatenate(records['depth']['ratios'], axis=0)
        records['depth']['rms'] = np.concatenate(records['depth']['rms'], axis=0)
        records['depth']['rms_log'] = np.concatenate(records['depth']['rms_log'], axis=0)
        records['depth']['rms_log'] = records['depth']['rms_log'][~np.isnan(records['depth']['rms_log'])]
        val_metrics['depth']['abs_err'] = (records['depth']['abs_errs'] * np.array(batch_size)).sum() / sum(batch_size)
        val_metrics['depth']['rel_err'] = (records['depth']['rel_errs'] * np.array(batch_size)).sum() / sum(batch_size)
        val_metrics['depth']['sq_rel_err'] = (records['depth']['sq_rel_errs'] * np.array(batch_size)).sum() / sum(
            batch_size)
        val_metrics['depth']['sigma_1.25'] = np.mean(np.less_equal(records['depth']['ratios'], 1.25)) * 100
        val_metrics['depth']['sigma_1.25^2'] = np.mean(np.less_equal(records['depth']['ratios'], 1.25 ** 2)) * 100
        val_metrics['depth']['sigma_1.25^3'] = np.mean(np.less_equal(records['depth']['ratios'], 1.25 ** 3)) * 100
        val_metrics['depth']['rms'] = (np.sum(records['depth']['rms']) / len(records['depth']['rms'])) ** 0.5
        # val_metrics['depth']['rms_log'] = (np.sum(records['depth']['rms_log']) / len(records['depth']['rms_log'])) ** 0.5

    if 'keypoint' in tasks:
        val_metrics['keypoint'] = {}
        val_metrics['keypoint']['err'] = (np.array(records['keypoint']['errs']) * np.array(batch_size)).sum() / sum(
            batch_size)

    if 'edge' in tasks:
        val_metrics['edge'] = {}
        val_metrics['edge']['err'] = (np.array(records['edge']['errs']) * np.array(batch_size)).sum() / sum(
            batch_size)

    return val_metrics## Prepare dataloaders

In [11]:
def read_yaml(input_args):
    # read in yaml
    parser = argparse.ArgumentParser()
    parser.add_argument("--config", required=True, help="Path for the config file")
    parser.add_argument("--exp_ids", type=int, nargs='+', default=[0], help="Path for the config file")
    parser.add_argument("--gpus", type=int, nargs='+', default=[0], help="Path for the config file")
    parser.add_argument("--cpu", default=False, action="store_true",  help="CPU instead of GPU")
    args = parser.parse_args(input_args)

    print(vars(args))
    # torch.cuda.set_device(args.gpu)
    with open(args.config) as f:
        opt = yaml.load(f)
    opt['cpu'] = args.cpu
    return opt, args.gpus, args.exp_ids

In [12]:
# input_args = " --config  yamls/adashare/nyu_v2_3task_test.yml --cpu ".split()
input_args = " --config yamls/adashare/nyu_v2_2task.yml --cpu ".split()
# pretrained_path = "/mnt/f/models_adashare/NYU_v2_3Task_pretrain"
print(input_args)
# print(pretrained_path)

['--config', 'yamls/adashare/nyu_v2_2task.yml', '--cpu']


#  Train

## Create folders and print options 

In [13]:
# ********************************************************************
# ****************** create folders and print options ****************
# ********************************************************************
# read the yaml
print_separator('READ YAML')
opt, gpu_ids, _ = read_yaml(input_args)
fix_random_seed(opt["seed"][0])
create_path(opt)
# print yaml on the screen
lines = print_yaml(opt)
for line in lines: print(line)
    
# print to file
with open(os.path.join(opt['paths']['log_dir'], opt['exp_name'], 'opt.txt'), 'w+') as f:
    f.writelines(lines)

##################################################
####################READ YAML#####################
##################################################
{'config': 'yamls/adashare/nyu_v2_2task.yml', 'exp_ids': [0], 'gpus': [0], 'cpu': True}

exp_name. : nyu_v2_2task
seed. : [88, 45, 50, 100, 44, 48, 2048, 2222, 9999]
backbone. : ResNet18
tasks. : ['seg', 'sn']
lambdas. : [1, 20]
tasks_num_class. : [40, 3]
policy_model. : task-specific
paths.
paths.log_dir. : ../experiments/logs/
paths.result_dir. : ../experiments/results
paths.checkpoint_dir. : ../experiments/checkpoints
dataload.
dataload.dataset. : NYU_v2
dataload.dataroot. : /mnt/f/MLDatasets/nyu_v2
dataload.orig_dataroot. : /data/datasets/nyu_v2
dataload.crop_h. : 321
dataload.crop_w. : 321
policy. : True
init_neg_logits. : None
is_sparse. : True
is_sharing. : True
skip_layer. : 0
is_curriculum. : True
curriculum_speed. : 3
fix_BN. : False
diff_sparsity_weights. : True
retrain_from_pl. : False
train.
train.batch_size. : 16
train.to

<ipython-input-11-6ef41f77d514>:13: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  opt = yaml.load(f)


In [14]:
opt['dataload']

{'dataset': 'NYU_v2',
 'dataroot': '/mnt/f/MLDatasets/nyu_v2',
 'orig_dataroot': '/data/datasets/nyu_v2',
 'crop_h': 321,
 'crop_w': 321}

## Prepare dataloaders

In [7]:
    # ********************************************************************
    # ******************** Prepare the dataloaders ***********************
    # ********************************************************************
    # load the dataloader
    print_separator('CREATE DATALOADERS')
    
    if opt['dataload']['dataset'] == 'NYU_v2':
        # To warm up
        trainset  = NYU_v2(opt['dataload']['dataroot'], 'train', opt['dataload']['crop_h'], opt['dataload']['crop_w'])
        # To update the network parameters
        trainset1 = NYU_v2(opt['dataload']['dataroot'], 'train1', opt['dataload']['crop_h'], opt['dataload']['crop_w'])
        # To update the policy weights
        trainset2 = NYU_v2(opt['dataload']['dataroot'], 'train2', opt['dataload']['crop_h'], opt['dataload']['crop_w'])
        # To validate
        valset    = NYU_v2(opt['dataload']['dataroot'], 'test')
 
    else:
        raise NotImplementedError('Dataset %s is not implemented' % opt['dataload']['dataset'])
## Prepare dataloaders

##################################################
################CREATE DATALOADERS################
##################################################
NYU_v2
NYU_v2
NYU_v2
NYU_v2


In [8]:
IMG_MEAN = np.array((104.00698793, 116.66876762, 122.67891434), dtype=np.float32)
print(IMG_MEAN.shape, IMG_MEAN)
print(IMG_MEAN[np.newaxis, np.newaxis, :].shape, IMG_MEAN[np.newaxis, np.newaxis, :])
IMG_MEAN = np.tile(IMG_MEAN[np.newaxis, np.newaxis, :], (480, 640,1))
print(IMG_MEAN.shape) 
print(IMG_MEAN[:5,:5,0])
print(IMG_MEAN[:5,:5,1])
print(IMG_MEAN[:5,:5,2])


(3,) [104.00699 116.66877 122.67892]
(1, 1, 3) [[[104.00699 116.66877 122.67892]]]
(480, 640, 3)
[[104.00699 104.00699 104.00699 104.00699 104.00699]
 [104.00699 104.00699 104.00699 104.00699 104.00699]
 [104.00699 104.00699 104.00699 104.00699 104.00699]
 [104.00699 104.00699 104.00699 104.00699 104.00699]
 [104.00699 104.00699 104.00699 104.00699 104.00699]]
[[116.66877 116.66877 116.66877 116.66877 116.66877]
 [116.66877 116.66877 116.66877 116.66877 116.66877]
 [116.66877 116.66877 116.66877 116.66877 116.66877]
 [116.66877 116.66877 116.66877 116.66877 116.66877]
 [116.66877 116.66877 116.66877 116.66877 116.66877]]
[[122.67892 122.67892 122.67892 122.67892 122.67892]
 [122.67892 122.67892 122.67892 122.67892 122.67892]
 [122.67892 122.67892 122.67892 122.67892 122.67892]
 [122.67892 122.67892 122.67892 122.67892 122.67892]
 [122.67892 122.67892 122.67892 122.67892 122.67892]]


In [9]:
print('size of training set 0 (warm up)       : ', len(trainset))
print('size of training set 1 (network parms) : ', len(trainset1))
print('size of training set 2 (policy weights): ', len(trainset2))
print('size of validation set                 : ', len(valset))

size of training set 0 (warm up)       :  795
size of training set 1 (network parms) :  636
size of training set 2 (policy weights):  159
size of validation set                 :  654


In [10]:
train_loader  = DataLoader(trainset , batch_size=opt['train']['batch_size'], drop_last=True, num_workers=2, shuffle=True)
train1_loader = DataLoader(trainset1, batch_size=opt['train']['batch_size'], drop_last=True, num_workers=2, shuffle=True)
train2_loader = DataLoader(trainset2, batch_size=opt['train']['batch_size'], drop_last=True, num_workers=2, shuffle=True)
val_loader    = DataLoader(valset   , batch_size=opt['train']['batch_size'], drop_last=True, num_workers=2, shuffle=False)

In [11]:
print(f" batch size       : {opt['train']['batch_size']}")
print(f' len train_loader : {len(train_loader)}')
print(f' len train1_loader: {len(train1_loader)}')
print(f' len train2_loader: {len(train2_loader)}')
print(f' len val_loader   : {len(val_loader)}')

 batch size       : 16
 len train_loader : 49
 len train1_loader: 39
 len train2_loader: 9
 len val_loader   : 40


In [12]:
opt['train']['weight_iter_alternate'] = opt['train'].get('weight_iter_alternate', len(train1_loader))
opt['train']['alpha_iter_alternate'] = opt['train'].get('alpha_iter_alternate', len(train2_loader))

print(opt['train']['weight_iter_alternate'], opt['train']['alpha_iter_alternate'])

39 9


##  Create Environment

In [13]:
# ********************************************************************
# ********************Create the environment *************************
# ********************************************************************
# create the model and the pretrain model
print_separator('CREATE THE ENVIRONMENT')
environ = BlockDropEnv(opt['paths']['log_dir'], 
                       opt['paths']['checkpoint_dir'], 
                       opt['exp_name'],
                       opt['tasks_num_class'], 
                       opt['init_neg_logits'], 
                       gpu_ids[0],
                       opt['train']['init_temp'], 
                       opt['train']['decay_temp'], is_train=True, opt=opt)

##################################################
##############CREATE THE ENVIRONMENT##############
##################################################
BlockDropEnv
Create MTL2 with 
 block: <class 'models.base.BasicBlock'> 
 layers: [2, 2, 2, 2] 
 tasks_num_class: [40, 3] 
 init_method: equal
define the scheduler (not policy learning)


In [14]:
current_iter = 0
current_iter_w, current_iter_a = 0, 0
if opt['train']['resume']:
    print(' Resume training')
    current_iter = environ.load(opt['train']['which_iter'])
    environ.networks['mtl-net'].reset_logits()

In [15]:
environ.define_optimizer(policy_learning=False)
environ.define_scheduler(policy_learning=False)

define the scheduler (not policy learning)


In [16]:
if torch.cuda.is_available():
    environ.cuda(gpu_ids)
else:
    print('cuda not available')
    environ.cpu()

cuda not available
base_env.cpu()
 Network item mtl-net moved to cpu
blockdrop_env.cpu()


In [17]:
batch_enumerator = enumerate(train_loader)
batch_enumerator1 = enumerate(train1_loader)
batch_enumerator2 = enumerate(train2_loader)
flag = 'update_w'
environ.fix_alpha()
environ.free_w(opt['fix_BN'])
best_value, best_iter = 0, 0

In [18]:
if opt['dataload']['dataset'] == 'NYU_v2':
    if len(opt['tasks_num_class']) == 2:
        refer_metrics = {'seg': {'mIoU': 0.413, 'Pixel Acc': 0.691},
                         'sn': {'Angle Mean': 15, 'Angle Median': 11.5, 'Angle 11.25': 49.2, 'Angle 22.5': 76.7,
                                'Angle 30': 86.8}}
    elif len(opt['tasks_num_class']) == 3:
        refer_metrics = {'seg': {'mIoU': 0.275, 'Pixel Acc': 0.589},
                         'sn': {'Angle Mean': 17.5, 'Angle Median': 14.2, 'Angle 11.25': 34.9, 'Angle 22.5': 73.3,
                                'Angle 30': 85.7},
                         'depth': {'abs_err': 0.62, 'rel_err': 0.25, 'sigma_1.25': 57.9,
                                   'sigma_1.25^2': 85.8, 'sigma_1.25^3': 95.7}}
    else:
        raise ValueError('num_class = %d is invalid' % len(opt['tasks_num_class']))
else:
    raise NotImplementedError('Dataset %s is not implemented' % opt['dataload']['dataset'])

In [19]:
print(refer_metrics)
print(opt['backbone'])
print(f" Total iterations: {opt['train']['total_iters']}")
print(f" Warm-up iterations: {opt['train']['warm_up_iters']}")

{'seg': {'mIoU': 0.413, 'Pixel Acc': 0.691}, 'sn': {'Angle Mean': 15, 'Angle Median': 11.5, 'Angle 11.25': 49.2, 'Angle 22.5': 76.7, 'Angle 30': 86.8}}
ResNet18
 Total iterations: 20000
 Warm-up iterations: 4000


In [20]:
best_metrics = None
p_epoch = 0
flag_warmup = True

if opt['backbone'] == 'ResNet18':
    num_blocks = 8
elif opt['backbone'] in ['ResNet34', 'ResNet50']:
    num_blocks = 18
elif opt['backbone'] == 'ResNet101':
    num_blocks = 33
elif opt['backbone'] == 'WRN':
    num_blocks = 15
else:
    raise ValueError('Backbone %s is invalid' % opt['backbone'])

In [21]:
from datetime import datetime
datetime.now().strftime('%F %H:%M:%S:%f')

'2021-09-13 16:22:34:449787'

In [22]:
while current_iter < opt['train']['total_iters']:
    start_time = time.time()
    print(f" {datetime.now().strftime('%F %H:%M:%S:%f')} - iteration: {current_iter}")
    environ.train()
    current_iter += 1
    
    # warm up

    if current_iter < opt['train']['warm_up_iters']:
        
        batch_idx, batch = next(batch_enumerator)
        environ.set_inputs(batch)
        
        environ.optimize(opt['lambdas'], is_policy=False, flag='update_w')
        
        if batch_idx == len(train_loader) - 1:
            batch_enumerator = enumerate(train_loader)

        if should(current_iter, opt['train']['print_freq']):
            environ.print_loss(current_iter, start_time)
            environ.resize_results()

        # validation
        if should(current_iter, opt['train']['val_freq']):
            environ.eval()
            num_seg_class = opt['tasks_num_class'][opt['tasks'].index('seg')] if 'seg' in opt['tasks'] else -1
            val_metrics = eval(environ, val_loader, opt['tasks'], policy=False, num_train_layers=None, num_seg_cls=num_seg_class)
            environ.print_loss(current_iter, start_time, val_metrics)
            environ.save('latest', current_iter)
            environ.train()

    else:
        if flag_warmup:
            environ.define_optimizer(policy_learning=True)
            environ.define_scheduler(policy_learning=True)

            flag_warmup = False

        if current_iter == opt['train']['warm_up_iters']:
            environ.save('warmup', current_iter)
            environ.fix_alpha()

        #-----------------------------------------
        # Update the network weights
        #-----------------------------------------
        if flag == 'update_w':
            
            current_iter_w += 1
            batch_idx_w, batch = next(batch_enumerator1)
            environ.set_inputs(batch)

            if opt['is_curriculum']:
                num_train_layers = p_epoch // opt['curriculum_speed'] + 1
            else:
                num_train_layers = None

            environ.optimize(opt['lambdas'], is_policy=opt['policy'], flag=flag, num_train_layers=num_train_layers,
                             hard_sampling=opt['train']['hard_sampling'])

            if should(current_iter, opt['train']['print_freq']):
                environ.print_loss(current_iter, start_time)
                environ.resize_results()

            if should(current_iter_w, opt['train']['weight_iter_alternate']):
                flag = 'update_alpha'
                environ.fix_w()
                environ.free_alpha()
                # do the validation on the test set
                environ.eval()
                print('Evaluating...')
                num_seg_class = opt['tasks_num_class'][opt['tasks'].index('seg')] if 'seg' in opt['tasks'] else -1
                val_metrics = eval(environ, val_loader, opt['tasks'], policy=opt['policy'],
                                   num_train_layers=num_train_layers, hard_sampling=opt['train']['hard_sampling'],
                                   num_seg_cls=num_seg_class)
                environ.print_loss(current_iter, start_time, val_metrics)
                environ.save('latest', current_iter)

                if current_iter - opt['train']['warm_up_iters'] >= num_blocks * opt['curriculum_speed'] * \
                        (opt['train']['weight_iter_alternate'] + opt['train']['alpha_iter_alternate']):
                    new_value = 0

                    for k in refer_metrics.keys():
                        if k in val_metrics.keys():
                            for kk in val_metrics[k].keys():
                                if not kk in refer_metrics[k].keys():
                                    continue
                                if (k == 'sn' and kk in ['Angle Mean', 'Angle Median']) or (
                                        k == 'depth' and not kk.startswith('sigma')) or (kk == 'err'):
                                    value = refer_metrics[k][kk] / val_metrics[k][kk]
                                else:
                                    value = val_metrics[k][kk] / refer_metrics[k][kk]

                                value = value / len(list(set(val_metrics[k].keys()) & set(refer_metrics[k].keys())))
                                new_value += value

                    if new_value > best_value:
                        best_value = new_value
                        best_metrics = val_metrics
                        best_iter = current_iter
                        environ.save('best', current_iter)
                    print('new value: %.3f' % new_value)
                    print('best iter: %d, best_value: %.3f' % (best_iter, best_value), best_metrics)
                environ.train()

            if batch_idx_w == len(train1_loader) - 1:
                batch_enumerator1 = enumerate(train1_loader)
                
        #-----------------------------------------
        # update the policy network
        #-----------------------------------------
        elif flag == 'update_alpha':
            current_iter_a += 1
            batch_idx_a, batch = next(batch_enumerator2)
            environ.set_inputs(batch)
            if opt['is_curriculum']:
                num_train_layers = p_epoch // opt['curriculum_speed'] + 1
            else:
                num_train_layers = None

            environ.optimize(opt['lambdas'], is_policy=opt['policy'], flag=flag, num_train_layers=num_train_layers,
                             hard_sampling=opt['train']['hard_sampling'])

            if should(current_iter, opt['train']['print_freq']):
                environ.print_loss(current_iter, start_time)
                environ.resize_results()
                # environ.visual_policy(current_iter)

            if should(current_iter_a, opt['train']['alpha_iter_alternate']):
                flag = 'update_w'
                environ.fix_alpha()
                environ.free_w(opt['fix_BN'])
                environ.decay_temperature()
                # print the distribution
                dists = environ.get_policy_prob()
                print(np.concatenate(dists, axis=-1))
                p_epoch += 1

            if batch_idx_a == len(train2_loader) - 1:
                batch_enumerator2 = enumerate(train2_loader)

        else:
            raise ValueError('flag %s is not recognized' % flag)


 2021-09-13 16:22:40:044094 - iteration: 0
 2021-09-13 16:23:36:040962 - iteration: 1
 2021-09-13 16:24:40:076664 - iteration: 2
 2021-09-13 16:25:36:355366 - iteration: 3
 2021-09-13 16:26:31:331287 - iteration: 4
 2021-09-13 16:27:29:040309 - iteration: 5
 2021-09-13 16:28:27:601892 - iteration: 6
 2021-09-13 16:29:23:527433 - iteration: 7
 2021-09-13 16:30:21:045091 - iteration: 8
 2021-09-13 16:31:17:560061 - iteration: 9
 2021-09-13 16:32:13:252647 - iteration: 10
 2021-09-13 16:33:08:472513 - iteration: 11
 2021-09-13 16:34:04:040421 - iteration: 12
 2021-09-13 16:34:57:914646 - iteration: 13
 2021-09-13 16:35:53:499784 - iteration: 14
 2021-09-13 16:36:47:755138 - iteration: 15
 2021-09-13 16:37:44:207223 - iteration: 16
 2021-09-13 16:38:39:126548 - iteration: 17
 2021-09-13 16:39:34:257094 - iteration: 18
 2021-09-13 16:40:28:806056 - iteration: 19
 2021-09-13 16:41:24:330273 - iteration: 20
 2021-09-13 16:42:19:162600 - iteration: 21
 2021-09-13 16:43:13:806921 - iteration: 2

 2021-09-13 19:10:16:348803 - iteration: 181
 2021-09-13 19:11:13:799563 - iteration: 182
 2021-09-13 19:12:10:260383 - iteration: 183
 2021-09-13 19:13:07:598266 - iteration: 184
 2021-09-13 19:14:05:673744 - iteration: 185
 2021-09-13 19:15:05:235403 - iteration: 186
 2021-09-13 19:16:05:296143 - iteration: 187
 2021-09-13 19:17:04:322919 - iteration: 188
 2021-09-13 19:18:00:427744 - iteration: 189
 2021-09-13 19:18:57:200859 - iteration: 190
 2021-09-13 19:19:53:790155 - iteration: 191
 2021-09-13 19:20:57:721688 - iteration: 192
 2021-09-13 19:22:06:288556 - iteration: 193
 2021-09-13 19:23:03:306991 - iteration: 194
 2021-09-13 19:24:02:308190 - iteration: 195
 2021-09-13 19:24:59:339017 - iteration: 196
 2021-09-13 19:26:01:834515 - iteration: 197
 2021-09-13 19:26:58:957791 - iteration: 198
 2021-09-13 19:27:56:131716 - iteration: 199
-------------------------------------------------------------
seg:
update: 200, time: 59.275 total: 2.455 
sn:
update: 200, time: 59.276 total: 0

KeyboardInterrupt: 